In [1]:
import kdl_parser_py.urdf
import PyKDL as kdl
import numpy as np

ok, tree = kdl_parser_py.urdf.treeFromFile("./urdf/stretch_pruned.urdf")
chain = tree.getChain("base_link", "link_grasp_center")
print(f"joints: {chain.getNrOfJoints()}, links: {chain.getNrOfSegments()}")

The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
joints: 8, links: 12


In [19]:
def np_jacobian(J, fixed_joints=()):
    return np.array([[J[i,j] if j not in fixed_joints else 0 for j in range(J.columns())] for i in range(J.rows())])

q = kdl.JntArray(chain.getNrOfJoints())
for i, j in enumerate([0,1,0,0,0,1,1,0]):
    q[i] = j
jac_solver = kdl.ChainJntToJacSolver(chain)
J = kdl.Jacobian(chain.getNrOfJoints())
jac_solver.JntToJac(q,J)
print(J)
# J = np.array([[J[i,j] if j!=1 else 0 for j in range(J.columns())] for i in range(J.rows())])
# J = np.array([[J[i,j] for j in range(J.columns())] for i in range(J.rows())])
J = np_jacobian(J, fixed_joints=(1,))
J_pinv = np.linalg.pinv(J)
with np.printoptions(precision=4, suppress=True, linewidth=100):
    print(J)
    print(J_pinv)

           1      1.28793  1.10627e-06     0.841471     0.841471     0.841471     0.841471     0.111019
           0      1.96626 -5.07554e-06    -0.540302    -0.540302    -0.540302    -0.540302     0.172904
           0            0            1 -7.34642e-06 -7.34642e-06 -7.34642e-06 -7.34642e-06  7.54775e-07
           0            0            0            0            0            0            0  7.28806e-06
           0            0            0            0            0            0            0 -9.04483e-06
           0            1            0            0            0            0            0            1
[[ 1.      0.      0.      0.8415  0.8415  0.8415  0.8415  0.111 ]
 [ 0.      0.     -0.     -0.5403 -0.5403 -0.5403 -0.5403  0.1729]
 [ 0.      0.      1.     -0.     -0.     -0.     -0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.     -0.    ]
 [ 0.      0.      0.      0.      0.    

In [ ]:
from __future__ import annotations
import rospy
from sensor_msgs.msg import JointState


class joint_state_listener:
    def __init__(self) -> None:
        rospy.init_node("joint_state_listener", anonymous=True)
        rospy.Subscriber("/joint_states", JointState, self.joint_state_cb)

    def joint_state_cb(self, joint_states: JointState) -> None:
        self.msg: JointState = joint_states

    # publish rate is 50Hz in gazebo
    def get_latest_pos(self) -> tuple[rospy.Duration, tuple]:
        duration: rospy.Duration = rospy.get_rostime() - self.msg.header.stamp
        # name: - joint_arm_l0 - joint_arm_l1 - joint_arm_l2 - joint_arm_l3 - joint_gripper_finger_left
        #   - joint_gripper_finger_right - joint_head_pan - joint_head_tilt - joint_left_wheel
        #   - joint_lift - joint_right_wheel - joint_wrist_yaw
        q_modified = (
            0.0,
            0.0,
            0.0,
            0.0,
            self.msg.position[self.msg.name.index("joint_lift")],
            0.0,
            self.msg.position[self.msg.name.index("joint_arm_l3")],
            self.msg.position[self.msg.name.index("joint_arm_l2")],
            self.msg.position[self.msg.name.index("joint_arm_l1")],
            self.msg.position[self.msg.name.index("joint_arm_l0")],
            self.msg.position[self.msg.name.index("joint_wrist_yaw")],
            0.0,
            0.0,
        )
        return duration, q_modified


JointStateListener = joint_state_listener()